title: PWCB business case
author: Vladas Jankus 
date: 2021-06-18
region: EU  
tags: cards, card, product, pwcb, pay with cahsback, cash, cashback, atmsummary: Customers are able to cash out up to 200 Eur when buying over 20 Eur with certain merchants. This outlook focused on German market. Main objective was to quantify potential customers who could switch from ATM to PWCB. Main motivation is that PWCB cashouts do not have a cost for N26. Since the start of 2021, there were around 2k customers who used PWCB in Germany. There were 109k customers who visited these merchants, but did not use PWCB and cashed out at ATM 2 days before or after the visit. Out of these, about 30% comply with minimum purchase (20 Eur) or maximum cashout (200 Eur) requirement. Weekly, there are around 5k potential customers who don't use PWCB, but visit these merchants, spend over 20 EUR and then cashout < 200 EUR at ATMs. They make around 9k cashouts (at ATMs) weekly. Roughly calculated - if we could make 30% of potential customers to use PWCB, we could expect around 1.8k transactions and 150k EUR weekly to switch from ATM to PWCB.

In [1]:
import pandas as pd
import altair as alt
from utils.datalib_database import df_from_sql
from IPython.display import HTML, Markdown as md, display

/usr/local/lib/python3.7/site-packages/datalib/vault.py:40: FutureWarning: The default value of regex will change from True to False in a future version.
  r"(.*)\[(.*)]", r"\1"
/usr/local/lib/python3.7/site-packages/datalib/vault.py:42: FutureWarning: The default value of regex will change from True to False in a future version.
  vault_params["key"] = vault_params["index"].str.replace(r"(.*)\[(.*)]", r"\2")


In [2]:
top_merchants = df_from_sql(
    "redshiftreader",
    """
        with ad_processing_code_90000 as (
            select 
                ad.id, 
                ad.processing_code, 
                ad.transaction_amount
            from ad_transaction_requests ad
            where ad.created > '2021-01-01'
                and ad.processing_code = '90000'
                and ad.message_type = 'AUTHORIZATION'
        )
        select 
            case
                when tx.merchant_name in ('DM-Drogerie Markt', 'DM Drogerie Markt', 'DM-DROGERIE MARKT') then 'DM Drogerie Markt'
                when tx.merchant_name in ('Mueller GmbH & Co.KG', 'Mueller MH Handels Gmb', 'Mueller Holding GmbH &') then 'Mueller'
                when left(lower(tx.merchant_name), 6) = 'famila' then 'FAMILIA'
                else tx.merchant_name
            end as merchant_name,
            sum((ad.id is not null)::int) as pwcb_transactions,
            count(*) as transactions,
            round(pwcb_transactions::float / transactions * 100, 2)::text || '%' as pwcb_percentage
        from dbt.zrh_card_transactions tx
        left join ad_processing_code_90000 ad on ad.id = tx.id
        where tx.created > '2021-01-01'
            and tx.merchant_country = 'DEU'
            and (
                tx.merchant_name in (
                    'DM-Drogerie Markt', 'DM Drogerie Markt', 'DM-DROGERIE MARKT',
                    'Netto Marken-Discount',
                    'Mueller GmbH & Co.KG', 'Mueller MH Handels Gmb', 'Mueller Holding GmbH &',
                    'Norma',
                    'Globus SB-Warenhaus'
                    )
                    or left(lower(tx.merchant_name), 6) = 'famila'
                )
            and tx.type = 'AA'
        group by 1
        order by 2 desc
    """,
)

{"message": "started", "db": "redshiftreader", "name": "datalib-logger", "args": [], "levelname": "INFO", "pathname": "/usr/local/lib/python3.7/site-packages/datalib/database.py", "filename": "database.py", "module": "database", "lineno": 121, "funcName": "df_from_sql", "created": "20210617T050350", "processName": "MainProcess", "service": "fargo", "environment": "local", "loggerId": "58ed8bd6-d131-4ffa-a931-a5b250a884ca", "hostname": "172.19.0.4"}
{"message": "success", "db": "redshiftreader", "duration": 45.6609, "name": "datalib-logger", "args": [], "levelname": "INFO", "pathname": "/usr/local/lib/python3.7/site-packages/datalib/database.py", "filename": "database.py", "module": "database", "lineno": 131, "funcName": "df_from_sql", "created": "20210617T050436", "processName": "MainProcess", "service": "fargo", "environment": "local", "loggerId": "58ed8bd6-d131-4ffa-a931-a5b250a884ca", "hostname": "172.19.0.4"}


In [3]:
pwcb_amount_cst = df_from_sql(
    "redshiftreader",
    """
        with ad_processing_code_90000 as (
            select
                ad.id,
                ad.processing_code,
                ad.transaction_amount
            from ad_transaction_requests ad
            where ad.created > '2021-01-01'
              and ad.processing_code = '90000'
              and ad.message_type = 'AUTHORIZATION'
        ),
        merchant_transactions as (
            select
                user_created,
                created as tx_timestamp,
                (amount_cents_eur / 100.0) - 20 as pwcb_amount
            from dbt.zrh_card_transactions tx
            inner join ad_processing_code_90000 ad
                on ad.id = tx.id
            where tx.created > '2021-01-01'
              and tx.merchant_country = 'DEU'
              and (
                        tx.merchant_name in (
                                             'DM-Drogerie Markt', 'DM Drogerie Markt', 'DM-DROGERIE MARKT',
                                             'Netto Marken-Discount',
                                             'Mueller GmbH & Co.KG', 'Mueller MH Handels Gmb', 'Mueller Holding GmbH &',
                                             'Norma',
                                             'Globus SB-Warenhaus'
                        )
                    or left(lower(tx.merchant_name), 6) = 'famila'
                )
              and tx.type = 'AA'
        )
        select
            to_char(tx_timestamp, '"CW "iw') as pwcb_merchant_visit,
            count(distinct user_created) as customers,
            count(*) as transactions,
            sum(pwcb_amount) as sum_pwcb
        from merchant_transactions
        where to_char(tx_timestamp, 'iyyy-iw') >= '2021-01'
            and to_char(tx_timestamp, 'iyyy-iw') < to_char(current_date, 'iyyy-iw')
        group by 1
    """,
)

{"message": "started", "db": "redshiftreader", "name": "datalib-logger", "args": [], "levelname": "INFO", "pathname": "/usr/local/lib/python3.7/site-packages/datalib/database.py", "filename": "database.py", "module": "database", "lineno": 121, "funcName": "df_from_sql", "created": "20210617T050442", "processName": "MainProcess", "service": "fargo", "environment": "local", "loggerId": "58ed8bd6-d131-4ffa-a931-a5b250a884ca", "hostname": "172.19.0.4"}
{"message": "success", "db": "redshiftreader", "duration": 67.5809, "name": "datalib-logger", "args": [], "levelname": "INFO", "pathname": "/usr/local/lib/python3.7/site-packages/datalib/database.py", "filename": "database.py", "module": "database", "lineno": 131, "funcName": "df_from_sql", "created": "20210617T050550", "processName": "MainProcess", "service": "fargo", "environment": "local", "loggerId": "58ed8bd6-d131-4ffa-a931-a5b250a884ca", "hostname": "172.19.0.4"}


In [4]:
pwcb_atm = df_from_sql(
    "redshiftreader",
    """
        with ad_processing_code_90000 as (
            select
                ad.id,
                ad.processing_code,
                ad.transaction_amount
            from ad_transaction_requests ad
            where ad.created > '2021-01-01'
              and ad.processing_code = '90000'
              and ad.message_type = 'AUTHORIZATION'
        ),
        merchant_transactions as (
            select
                user_created,
                created as tx_timestamp,
                (amount_cents_eur / 100.0) - 20 as pwcb_amount
            from dbt.zrh_card_transactions tx
            inner join ad_processing_code_90000 ad
                on ad.id = tx.id
            where tx.created > '2021-01-01'
              and tx.merchant_country = 'DEU'
              and (
                        tx.merchant_name in (
                                             'DM-Drogerie Markt', 'DM Drogerie Markt', 'DM-DROGERIE MARKT',
                                             'Netto Marken-Discount',
                                             'Mueller GmbH & Co.KG', 'Mueller MH Handels Gmb', 'Mueller Holding GmbH &',
                                             'Norma',
                                             'Globus SB-Warenhaus'
                        )
                    or left(lower(tx.merchant_name), 6) = 'famila'
                )
              and tx.type = 'AA'
        ), atm_cashout as (
            select
                merchant_transactions.user_created,
                tx_timestamp,
                merchant_transactions.pwcb_amount,
                zrh_card_transactions.amount_cents_eur / 100.0 as atm_amount
            from merchant_transactions
            left join dbt.zrh_card_transactions
                      on merchant_transactions.user_created = zrh_card_transactions.user_created
                          and zrh_card_transactions.card_tx_type = 'atm'
                          and zrh_card_transactions.created >= merchant_transactions.tx_timestamp - interval '2 days'
                          and zrh_card_transactions.created <= merchant_transactions.tx_timestamp + interval '2 days'
        )
        select
            to_char(tx_timestamp, '"CW "iw') as pwcb_merchant_visit,
            case
                when atm_amount > 200 then 'ATM > 200'
                when atm_amount <= 200 then 'ATM <= 200'
                else 'Only PWCB'
            end as atm_tx_amount,
            count(*) as atm_transactions
        from atm_cashout
        where to_char(tx_timestamp, 'iyyy-iw') >= '2021-01'
            and to_char(tx_timestamp, 'iyyy-iw') < to_char(current_date, 'iyyy-iw')
        group by 1, 2
    """,
)

{"message": "started", "db": "redshiftreader", "name": "datalib-logger", "args": [], "levelname": "INFO", "pathname": "/usr/local/lib/python3.7/site-packages/datalib/database.py", "filename": "database.py", "module": "database", "lineno": 121, "funcName": "df_from_sql", "created": "20210617T050553", "processName": "MainProcess", "service": "fargo", "environment": "local", "loggerId": "58ed8bd6-d131-4ffa-a931-a5b250a884ca", "hostname": "172.19.0.4"}
{"message": "success", "db": "redshiftreader", "duration": 67.6359, "name": "datalib-logger", "args": [], "levelname": "INFO", "pathname": "/usr/local/lib/python3.7/site-packages/datalib/database.py", "filename": "database.py", "module": "database", "lineno": 131, "funcName": "df_from_sql", "created": "20210617T050701", "processName": "MainProcess", "service": "fargo", "environment": "local", "loggerId": "58ed8bd6-d131-4ffa-a931-a5b250a884ca", "hostname": "172.19.0.4"}


In [5]:
potential_customers_15days = df_from_sql(
    "redshiftreader",
    """
        with ad_processing_code_90000 as (
            select
                ad.id,
                ad.processing_code,
                ad.transaction_amount
            from ad_transaction_requests ad
            where ad.created > '2021-01-01'
              and ad.processing_code = '90000'
              and ad.message_type = 'AUTHORIZATION'
        ),
        merchant_transactions as (
            select
                user_created,
                created as tx_timestamp,
                ad.id is not null as pwcb_transaction
            from dbt.zrh_card_transactions tx
            left join ad_processing_code_90000 ad
                on ad.id = tx.id
            where tx.created > '2021-01-01'
              and tx.merchant_country = 'DEU'
              and (
                        tx.merchant_name in (
                                             'DM-Drogerie Markt', 'DM Drogerie Markt', 'DM-DROGERIE MARKT',
                                             'Netto Marken-Discount',
                                             'Mueller GmbH & Co.KG', 'Mueller MH Handels Gmb', 'Mueller Holding GmbH &',
                                             'Norma',
                                             'Globus SB-Warenhaus'
                        )
                    or left(lower(tx.merchant_name), 6) = 'famila'
                )
              and tx.type = 'AA'
        ), potential_customers as (
            select
                merchant_transactions.user_created,
                merchant_transactions.pwcb_transaction,
                sum((zrh_card_transactions.user_created is not null)::int) > 0 as atm_transaction
            from merchant_transactions
            left join dbt.zrh_card_transactions
                      on merchant_transactions.user_created = zrh_card_transactions.user_created
                          and zrh_card_transactions.card_tx_type = 'atm'
                          and zrh_card_transactions.created >= merchant_transactions.tx_timestamp - interval '15 days'
                          and zrh_card_transactions.created <= merchant_transactions.tx_timestamp + interval '15 days'
            group by 1, 2
        ), customers_aggregated as (
            select
                user_created,
                listagg(
                    case
                        when pwcb_transaction then 'PWCB'
                        when not pwcb_transaction then 'No PWCB'
                    end
                    || ' ' ||
                    case
                        when atm_transaction then 'ATM'
                        when not atm_transaction then 'No ATM'
                    end
                    ) as cash_out_event
            from potential_customers
            group by 1
        )
        select
            case
                when cash_out_event in (
                                        'No PWCB ATMPWCB ATM', 'PWCB ATMNo PWCB ATM',
                                        'PWCB ATMNo PWCB No ATM', 'PWCB ATM'
                                       )
                    then 'PWCB and ATM'
                when cash_out_event in (
                                        'PWCB No ATM', 'No PWCB ATMPWCB No ATM',
                                        'No PWCB No ATMPWCB No ATM', 'PWCB No ATMNo PWCB No ATM'
                                       )
                    then 'PWCB'
                when cash_out_event in ('No PWCB ATM')
                    then 'ATM'
            end as actions,
            count(*)
        from customers_aggregated
        where cash_out_event != 'No PWCB No ATM'
        group by 1
    """,
)

{"message": "started", "db": "redshiftreader", "name": "datalib-logger", "args": [], "levelname": "INFO", "pathname": "/usr/local/lib/python3.7/site-packages/datalib/database.py", "filename": "database.py", "module": "database", "lineno": 121, "funcName": "df_from_sql", "created": "20210617T050702", "processName": "MainProcess", "service": "fargo", "environment": "local", "loggerId": "58ed8bd6-d131-4ffa-a931-a5b250a884ca", "hostname": "172.19.0.4"}
{"message": "success", "db": "redshiftreader", "duration": 21.6046, "name": "datalib-logger", "args": [], "levelname": "INFO", "pathname": "/usr/local/lib/python3.7/site-packages/datalib/database.py", "filename": "database.py", "module": "database", "lineno": 131, "funcName": "df_from_sql", "created": "20210617T050724", "processName": "MainProcess", "service": "fargo", "environment": "local", "loggerId": "58ed8bd6-d131-4ffa-a931-a5b250a884ca", "hostname": "172.19.0.4"}


In [6]:
potential_customers_2days = df_from_sql(
    "redshiftreader",
    """
        with ad_processing_code_90000 as (
            select
                ad.id,
                ad.processing_code,
                ad.transaction_amount
            from ad_transaction_requests ad
            where ad.created > '2021-01-01'
              and ad.processing_code = '90000'
              and ad.message_type = 'AUTHORIZATION'
        ),
        merchant_transactions as (
            select
                user_created,
                created as tx_timestamp,
                ad.id is not null as pwcb_transaction
            from dbt.zrh_card_transactions tx
            left join ad_processing_code_90000 ad
                on ad.id = tx.id
            where tx.created > '2021-01-01'
              and tx.merchant_country = 'DEU'
              and (
                        tx.merchant_name in (
                                             'DM-Drogerie Markt', 'DM Drogerie Markt', 'DM-DROGERIE MARKT',
                                             'Netto Marken-Discount',
                                             'Mueller GmbH & Co.KG', 'Mueller MH Handels Gmb', 'Mueller Holding GmbH &',
                                             'Norma',
                                             'Globus SB-Warenhaus'
                        )
                    or left(lower(tx.merchant_name), 6) = 'famila'
                )
              and tx.type = 'AA'
        ), potential_customers as (
            select
                merchant_transactions.user_created,
                merchant_transactions.pwcb_transaction,
                sum((zrh_card_transactions.user_created is not null)::int) > 0 as atm_transaction
            from merchant_transactions
            left join dbt.zrh_card_transactions
                      on merchant_transactions.user_created = zrh_card_transactions.user_created
                          and zrh_card_transactions.card_tx_type = 'atm'
                          and zrh_card_transactions.created >= merchant_transactions.tx_timestamp - interval '2 days'
                          and zrh_card_transactions.created <= merchant_transactions.tx_timestamp + interval '2 days'
            group by 1, 2
        ), customers_aggregated as (
            select
                user_created,
                listagg(
                    case
                        when pwcb_transaction then 'PWCB'
                        when not pwcb_transaction then 'No PWCB'
                    end
                    || ' ' ||
                    case
                        when atm_transaction then 'ATM'
                        when not atm_transaction then 'No ATM'
                    end
                    ) as cash_out_event
            from potential_customers
            group by 1
        )
        select
            case
                when cash_out_event in (
                                        'No PWCB ATMPWCB ATM', 'PWCB ATMNo PWCB ATM',
                                        'PWCB ATMNo PWCB No ATM', 'PWCB ATM'
                                       )
                    then 'PWCB and ATM'
                when cash_out_event in (
                                        'PWCB No ATM', 'No PWCB ATMPWCB No ATM',
                                        'No PWCB No ATMPWCB No ATM', 'PWCB No ATMNo PWCB No ATM'
                                       )
                    then 'PWCB'
                when cash_out_event in ('No PWCB ATM')
                    then 'ATM'
            end as actions,
            count(*)
        from customers_aggregated
        where cash_out_event != 'No PWCB No ATM'
        group by 1
    """,
)

{"message": "started", "db": "redshiftreader", "name": "datalib-logger", "args": [], "levelname": "INFO", "pathname": "/usr/local/lib/python3.7/site-packages/datalib/database.py", "filename": "database.py", "module": "database", "lineno": 121, "funcName": "df_from_sql", "created": "20210617T050729", "processName": "MainProcess", "service": "fargo", "environment": "local", "loggerId": "58ed8bd6-d131-4ffa-a931-a5b250a884ca", "hostname": "172.19.0.4"}
{"message": "success", "db": "redshiftreader", "duration": 17.4597, "name": "datalib-logger", "args": [], "levelname": "INFO", "pathname": "/usr/local/lib/python3.7/site-packages/datalib/database.py", "filename": "database.py", "module": "database", "lineno": 131, "funcName": "df_from_sql", "created": "20210617T050746", "processName": "MainProcess", "service": "fargo", "environment": "local", "loggerId": "58ed8bd6-d131-4ffa-a931-a5b250a884ca", "hostname": "172.19.0.4"}


In [7]:
cst_product = df_from_sql(
    "redshiftreader",
    """
        with ad_processing_code_90000 as (
            select
                ad.id,
                ad.processing_code,
                ad.transaction_amount
            from ad_transaction_requests ad
            where ad.created > '2021-01-01'
              and ad.processing_code = '90000'
              and ad.message_type = 'AUTHORIZATION'
        ),
        merchant_transactions as (
            select
                user_created,
                created as tx_timestamp,
                ad.id is not null as pwcb_transaction,
                amount_cents_eur >= 2000 as purchase_over_20_eur
            from dbt.zrh_card_transactions tx
            left join ad_processing_code_90000 ad
                on ad.id = tx.id
            where tx.created > '2021-01-01'
              and tx.merchant_country = 'DEU'
              and (
                        tx.merchant_name in (
                                             'DM-Drogerie Markt', 'DM Drogerie Markt', 'DM-DROGERIE MARKT',
                                             'Netto Marken-Discount',
                                             'Mueller GmbH & Co.KG', 'Mueller MH Handels Gmb', 'Mueller Holding GmbH &',
                                             'Norma',
                                             'Globus SB-Warenhaus'
                        )
                    or left(lower(tx.merchant_name), 6) = 'famila'
                )
              and tx.type = 'AA'
        ), potential_customers as (
            select
                merchant_transactions.user_created,
                merchant_transactions.pwcb_transaction,
                merchant_transactions.purchase_over_20_eur,
                zrh_card_transactions.amount_cents_eur <= 20000 as atm_less_than_200,
                sum((zrh_card_transactions.user_created is not null)::int) > 0 as atm_transaction
            from merchant_transactions
            left join dbt.zrh_card_transactions
                      on merchant_transactions.user_created = zrh_card_transactions.user_created
                          and zrh_card_transactions.card_tx_type = 'atm'
                          and zrh_card_transactions.created >= merchant_transactions.tx_timestamp - interval '2 days'
                          and zrh_card_transactions.created <= merchant_transactions.tx_timestamp + interval '2 days'
            group by 1, 2, 3, 4
        ), customers_aggregated as (
            select
                user_created,
                zrh_users.product_id,
                zrh_users.is_premium,
                purchase_over_20_eur,
                atm_less_than_200,
                listagg(
                    case
                        when pwcb_transaction then 'PWCB'
                        when not pwcb_transaction then 'No PWCB'
                    end
                    || ' ' ||
                    case
                        when atm_transaction then 'ATM'
                        when not atm_transaction then 'No ATM'
                    end
                    ) as cash_out_event
            from potential_customers
            left join dbt.zrh_users
                using(user_created)
            group by 1, 2, 3, 4, 5
        )
        select
            case
                when product_id in ('STANDARD', 'BUSINESS_CARD') or product_id is null then 'Standard'
                when is_premium then 'Premium'
                else 'Flex'
            end as subscription,
            case
                when cash_out_event = ('No PWCB ATM')  then 'ATM'
                else 'PWCB'
            end as actions,
            count(*)
        from customers_aggregated
        where cash_out_event != 'No PWCB No ATM'
            and (
                cash_out_event != 'No PWCB ATM'
                or (purchase_over_20_eur and atm_less_than_200)
            )
        group by 1, 2
    """,
)

{"message": "started", "db": "redshiftreader", "name": "datalib-logger", "args": [], "levelname": "INFO", "pathname": "/usr/local/lib/python3.7/site-packages/datalib/database.py", "filename": "database.py", "module": "database", "lineno": 121, "funcName": "df_from_sql", "created": "20210617T050747", "processName": "MainProcess", "service": "fargo", "environment": "local", "loggerId": "58ed8bd6-d131-4ffa-a931-a5b250a884ca", "hostname": "172.19.0.4"}
{"message": "success", "db": "redshiftreader", "duration": 19.6779, "name": "datalib-logger", "args": [], "levelname": "INFO", "pathname": "/usr/local/lib/python3.7/site-packages/datalib/database.py", "filename": "database.py", "module": "database", "lineno": 131, "funcName": "df_from_sql", "created": "20210617T050807", "processName": "MainProcess", "service": "fargo", "environment": "local", "loggerId": "58ed8bd6-d131-4ffa-a931-a5b250a884ca", "hostname": "172.19.0.4"}


In [8]:
cst_base = df_from_sql(
    "redshiftreader",
    """
        select 
            case
                when product_id in ('STANDARD', 'BUSINESS_CARD') or product_id is null then 'Standard'
                when is_premium then 'Premium'
                else 'Flex'
            end as subscription,
            'MAUs' as actions,
            count(*)
        from dbt.zrh_users 
        where closed_at is null and is_mau
        group by 1, 2
    """,
)

{"message": "started", "db": "redshiftreader", "name": "datalib-logger", "args": [], "levelname": "INFO", "pathname": "/usr/local/lib/python3.7/site-packages/datalib/database.py", "filename": "database.py", "module": "database", "lineno": 121, "funcName": "df_from_sql", "created": "20210617T050811", "processName": "MainProcess", "service": "fargo", "environment": "local", "loggerId": "58ed8bd6-d131-4ffa-a931-a5b250a884ca", "hostname": "172.19.0.4"}
{"message": "success", "db": "redshiftreader", "duration": 13.3235, "name": "datalib-logger", "args": [], "levelname": "INFO", "pathname": "/usr/local/lib/python3.7/site-packages/datalib/database.py", "filename": "database.py", "module": "database", "lineno": 131, "funcName": "df_from_sql", "created": "20210617T050824", "processName": "MainProcess", "service": "fargo", "environment": "local", "loggerId": "58ed8bd6-d131-4ffa-a931-a5b250a884ca", "hostname": "172.19.0.4"}


In [9]:
pot_cst_dd = df_from_sql(
    "redshiftreader",
    """
        with ad_processing_code_90000 as (
            select
                ad.id,
                ad.processing_code,
                ad.transaction_amount
            from ad_transaction_requests ad
            where ad.created > '2021-01-01'
              and ad.processing_code = '90000'
              and ad.message_type = 'AUTHORIZATION'
        ),
        merchant_transactions as (
            select
                user_created,
                created as tx_timestamp,
                amount_cents_eur >= 2000 as purchase_over_20_eur
            from dbt.zrh_card_transactions tx
            left join ad_processing_code_90000 ad
                on ad.id = tx.id
            where tx.created > '2021-01-01'
              and tx.merchant_country = 'DEU'
              and (
                        tx.merchant_name in (
                                             'DM-Drogerie Markt', 'DM Drogerie Markt', 'DM-DROGERIE MARKT',
                                             'Netto Marken-Discount',
                                             'Mueller GmbH & Co.KG', 'Mueller MH Handels Gmb', 'Mueller Holding GmbH &',
                                             'Norma',
                                             'Globus SB-Warenhaus'
                        )
                    or left(lower(tx.merchant_name), 6) = 'famila'
                )
              and tx.type = 'AA'
              and ad.id is null
        ), potential_customers as (
            select
                merchant_transactions.user_created,
                merchant_transactions.tx_timestamp,
                merchant_transactions.purchase_over_20_eur,
                zrh_card_transactions.amount_cents_eur <= 20000 as atm_less_than_200
            from merchant_transactions
            inner join dbt.zrh_card_transactions
                      on merchant_transactions.user_created = zrh_card_transactions.user_created
                          and zrh_card_transactions.card_tx_type = 'atm'
                          and zrh_card_transactions.created >= merchant_transactions.tx_timestamp - interval '2 days'
                          and zrh_card_transactions.created <= merchant_transactions.tx_timestamp + interval '2 days'
        )
        select
            to_char(tx_timestamp, '"CW "iw') as pwcb_merchant_visit,
            case
                when atm_less_than_200 then
                    case
                        when purchase_over_20_eur then 'Potential PWCB'
                        else 'Purchase under 20'
                    end
                else 'ATM over 200'
            end as situation,
            count(*)
        from potential_customers
        where to_char(tx_timestamp, 'iyyy-iw') >= '2021-01'
            and to_char(tx_timestamp, 'iyyy-iw') < to_char(current_date, 'iyyy-iw')
        group by 1, 2
    """,
)

{"message": "started", "db": "redshiftreader", "name": "datalib-logger", "args": [], "levelname": "INFO", "pathname": "/usr/local/lib/python3.7/site-packages/datalib/database.py", "filename": "database.py", "module": "database", "lineno": 121, "funcName": "df_from_sql", "created": "20210617T050826", "processName": "MainProcess", "service": "fargo", "environment": "local", "loggerId": "58ed8bd6-d131-4ffa-a931-a5b250a884ca", "hostname": "172.19.0.4"}
{"message": "success", "db": "redshiftreader", "duration": 19.0065, "name": "datalib-logger", "args": [], "levelname": "INFO", "pathname": "/usr/local/lib/python3.7/site-packages/datalib/database.py", "filename": "database.py", "module": "database", "lineno": 131, "funcName": "df_from_sql", "created": "20210617T050845", "processName": "MainProcess", "service": "fargo", "environment": "local", "loggerId": "58ed8bd6-d131-4ffa-a931-a5b250a884ca", "hostname": "172.19.0.4"}


In [10]:
pot_cst_volume = df_from_sql(
    "redshiftreader",
    """
        with ad_processing_code_90000 as (
            select
                ad.id,
                ad.processing_code,
                ad.transaction_amount
            from ad_transaction_requests ad
            where ad.created > '2021-01-01'
              and ad.processing_code = '90000'
              and ad.message_type = 'AUTHORIZATION'
        ),
        merchant_transactions as (
            select
                user_created,
                created as tx_timestamp
            from dbt.zrh_card_transactions tx
            left join ad_processing_code_90000 ad
                on ad.id = tx.id
            where tx.created > '2021-01-01'
              and tx.merchant_country = 'DEU'
              and (
                        tx.merchant_name in (
                                             'DM-Drogerie Markt', 'DM Drogerie Markt', 'DM-DROGERIE MARKT',
                                             'Netto Marken-Discount',
                                             'Mueller GmbH & Co.KG', 'Mueller MH Handels Gmb', 'Mueller Holding GmbH &',
                                             'Norma',
                                             'Globus SB-Warenhaus'
                        )
                    or left(lower(tx.merchant_name), 6) = 'famila'
                )
              and tx.type = 'AA'
              and ad.id is null
              and amount_cents_eur >= 2000
        ), potential_customers as (
            select
                merchant_transactions.user_created,
                merchant_transactions.tx_timestamp,
                zrh_card_transactions.amount_cents_eur / 100.0 as atm_volume
            from merchant_transactions
            inner join dbt.zrh_card_transactions
                      on merchant_transactions.user_created = zrh_card_transactions.user_created
                          and zrh_card_transactions.card_tx_type = 'atm'
                          and zrh_card_transactions.created >= merchant_transactions.tx_timestamp - interval '2 days'
                          and zrh_card_transactions.created <= merchant_transactions.tx_timestamp + interval '2 days'
                          and amount_cents_eur <= 20000
        )
        select
            to_char(tx_timestamp, '"CW "iw') as pwcb_merchant_visit,
            sum(atm_volume),
            count(*) as atm_transactions
        from potential_customers
        where to_char(tx_timestamp, 'iyyy-iw') >= '2021-01'
            and to_char(tx_timestamp, 'iyyy-iw') < to_char(current_date, 'iyyy-iw')
        group by 1
    """,
)

{"message": "started", "db": "redshiftreader", "name": "datalib-logger", "args": [], "levelname": "INFO", "pathname": "/usr/local/lib/python3.7/site-packages/datalib/database.py", "filename": "database.py", "module": "database", "lineno": 121, "funcName": "df_from_sql", "created": "20210617T050848", "processName": "MainProcess", "service": "fargo", "environment": "local", "loggerId": "58ed8bd6-d131-4ffa-a931-a5b250a884ca", "hostname": "172.19.0.4"}
{"message": "success", "db": "redshiftreader", "duration": 30.4677, "name": "datalib-logger", "args": [], "levelname": "INFO", "pathname": "/usr/local/lib/python3.7/site-packages/datalib/database.py", "filename": "database.py", "module": "database", "lineno": 131, "funcName": "df_from_sql", "created": "20210617T050918", "processName": "MainProcess", "service": "fargo", "environment": "local", "loggerId": "58ed8bd6-d131-4ffa-a931-a5b250a884ca", "hostname": "172.19.0.4"}


# Pay with cashback (PWCB) numbers for business case
Vladas Jankus<br/>
2021-06-16

### Contents:
* [1. TL;DR](#1)
* [2. PWCB Merchants](#2)
* [3. Potential Customers](#3)
* [4. PWCB user deep-dive](#4)
* [5. Potential customer deep-dive](#5)
* [6. Customer subscriptions](#6)
* [7. Conclusion](#7)

## 1. TL;DR <a class="anchor" id="1"></a>

Customers are able to cash out up to 200 Eur when buying over 20 Eur with certain merchants. This outlook focused on German market. Main objective was to quantify potential customers who could switch from ATM to PWCB. Main motivation is that PWCB cashouts do not have a cost for N26.

Since the start of 2021, there were around 2k customers who used PWCB in Germany. There were 109k customers who visited these merchants, but did not use PWCB and cashed out at ATM 2 days before or after the visit. Out of these, about 30% comply with minimum purchase (20 Eur) or maximum cashout (200 Eur) requirement.

Weekly, there are around 5k potential customers who don't use PWCB, but visit these merchants, spend over 20 EUR and then cashout < 200 EUR at ATMs. They make around 9k cashouts (at ATMs) weekly. Roughly calculated - if we could make 30% of potential customers to use PWCB, we could expect around 1.8k transactions and 150k EUR weekly to switch from ATM to PWCB.

## 2. PWCB Merchants <a class="anchor" id="2"></a>

First let's look at how many PWCB transactions we do have since the start of 2021. Below is a table of top merchants in terms of PWCB transactions. Some merchant names that have distinct store names were grouped together (e.g. DM versions are grouped under one name).

Since we are focusing on DEU market, it's only DEU merchants taken in here.

In [11]:
print("Table1: German PWCB merchants")
top_merchants.style.hide_index()

Table1: German PWCB merchants


merchant_name,pwcb_transactions,transactions,pwcb_percentage
DM Drogerie Markt,1995,767376,0.26%
Netto Marken-Discount,1390,767304,0.18%
Mueller,117,104892,0.11%
FAMILIA,112,39909,0.28%
Norma,75,86735,0.09%
Globus SB-Warenhaus,56,40725,0.14%


Table also includes the count of non-PWCB transactions in 2021. PWCB in all of them make up less than 1%, but there are differences in ratios. Possibly there are opportunities in raising awareness that customers can cash out in some of these merchants.

## 3. Potential customers  <a class="anchor" id="3"></a>

Let's look at how many customers we have who visited these merchants did not use PWCB, but did an ATM transaction 15 days before or 15 days after. Data is since 2021-01-01. See table below:

In [12]:
print(
    "Table2: Customers who went to PWCB merchants in 2021 and used ATM 15 days around the visit"
)
potential_customers_15days.style.hide_index()

Table2: Customers who went to PWCB merchants in 2021 and used ATM 15 days around the visit


actions,count
PWCB and ATM,1444
ATM,164052
PWCB,828


15 Days around the merchant visit appears to be too large of a period. 1444 customers who used PWCB also used an ATM and only 828 customers used only PWCB. There were 164k customers, who went to an ATM 15 days around their purchase with a PWCB merchant.

It's difficult to claim, that a customer who went to an ATM several days later could have cashed out at PWCB merchant. Therefore I would suggest to cut the 15 day period down to 2 days. See table below: <a class="anchor" id="table3"></a>

In [13]:
print(
    "Table3: Customers who went to PWCB merchants in 2021 and used ATM 2 days around the visit"
)
potential_customers_2days.style.hide_index()

Table3: Customers who went to PWCB merchants in 2021 and used ATM 2 days around the visit


actions,count
PWCB and ATM,606
PWCB,1666
ATM,115330


A lot less customers who used PWCB also went to ATM, which shows us that +/-2 day window is a lot more realistic. 

There were 115k customers who went to an ATM 2 days around purchasing with a PWCB merchant. This number includes the target customers and is broken down further in [potential customer deep dive](#5).

## 4. PWCB user deep-dive <a class="anchor" id="4"></a>

In this section we will briefly look at customers who are already using PWCB. A chart below displays unique customer count and volumes they used with PWCB. Please note that it's not possible to distinguish the exact amount cashed out from the whole PWCB transaction, so for estimation purposes I have deducted the minimum 20 EUR, and left the rest as a cash out.

In [14]:
source = pwcb_amount_cst

base = alt.Chart(source).encode(
    alt.X("pwcb_merchant_visit:O", axis=alt.Axis(title=None))
)

column = base.mark_bar(opacity=0.7, color="#5677A4").encode(
    alt.Y(
        "transactions",
        axis=alt.Axis(title="Weekly transactions", titleColor="#5677A4"),
        scale=alt.Scale(domain=[0, 300]),
    )
)

line = base.mark_line(stroke="#E68B39", interpolate="monotone").encode(
    alt.Y(
        "sum_pwcb",
        axis=alt.Axis(title="Weekly amount €", titleColor="#E68B39"),
        scale=alt.Scale(domain=[0, 23000]),
    )
)

alt.layer(column, line).resolve_scale(y="independent").properties(
    width=800, height=300, title="PWCB transaction count and volume"
)

alt.LayerChart(...)

In recent weeks there are around 150-200 unique customers (not on chart) who do around 200 PWCB transactions weekly. The approximate amount is between 17k EUR weekly. This makes it around 85 EUR for a transaction weekly. 

Note, that this is the PWCB transaction minus minimum 20 EUR needed for purchase. Cash withdrawal is just an approximation since we cannot accurately split the cashout value.

Some of the customers using PWCB still do ATM transactions 4 days around. Let's just look at the amounts they cash out at ATMs below.

In [15]:
alt.Chart(pwcb_atm).mark_area().encode(
    x=alt.X(
        "pwcb_merchant_visit:O",
        title=None,
    ),
    y=alt.Y(
        "atm_transactions:Q",
        # scale=alt.Scale(domain=(0, 50000)),
        title="ATM transaction count",
    ),
    color="atm_tx_amount:N",
).properties(
    width=800,
    height=300,
    title="ATM visits by PWCB customers (2 days around PWCB purchase)",
)

alt.Chart(...)

Only a small amount of PWCB customers do >200 EUR cashouts at ATMs. Approximately 30%-40% of money cashed out 2 days around PWCB visit is still coming from ATM transactions that are <200 EUR and could be eligible for PWCB. 

This makes sense because we cannot expect customers to cash out only at PWCB. From this chart we see that customers aware of PWCB cash out about 60% using this way.

## 5. Potential customer deep dive <a class="anchor" id="5"></a>

The fact that there was 115k customers in 2021 (from [Table3](#table3) above) who visited PWCB merchant but did cash out there does not really tell the full picture. Let's look at how many transactions these customers make with PWCB merchants weekly.

Chart below displays only these customers, who <b>did not make a PWCB, but had an ATM transaction</b> 2 days around visiting the merchant. Data is also aggregated weekly since the start of 2021.

In [16]:
alt.Chart(pot_cst_dd).mark_area().encode(
    x=alt.X(
        "pwcb_merchant_visit:O",
        title=None,
    ),
    y=alt.Y(
        "count:Q",
        scale=alt.Scale(domain=(0, 50000)),
        title="Visits",
    ),
    color="situation:N",
).properties(
    width=800,
    height=300,
    title="Visits to merchants without doing PWCB but doing an ATM (2 days around)",
)

alt.Chart(...)

It can be seen that the number of customers is increasing since the beginning of the year. Recently there has been around 50k transactions weekly. These are made by around 18k distinct customers (not displayed on chart).

<b style="color:#D4605B">Most of the customers</b> don't make purchases over 20 EUR, so they would not really be eligible for PWCB unless deliberately purchase more. Purchases under 20 EUR make up around 60% of transactions. There's also <b style="color:#5677A4">a small part of customers</b> who used ATM for over 200 EUR. That's usually around 10% of customers.

<b style="color:#E68B39">True potential customers</b> who made over 20 EUR purchases and used ATM under 200 EUR make up just a bit under 30% of the total. <i><b>That is around 5000 distinct customers weekly who make around 9000 visits to PWCB merchants.</b></i>

In [17]:
base = alt.Chart(pot_cst_volume).encode(
    alt.X("pwcb_merchant_visit:O", axis=alt.Axis(title=None))
)

line1 = base.mark_line(stroke="#5677A4", interpolate="monotone").encode(
    alt.Y("sum", axis=alt.Axis(title="ATM volume", titleColor="#5677A4"))
)

line2 = base.mark_line(stroke="#E68B39", interpolate="monotone").encode(
    alt.Y(
        "atm_transactions",
        axis=alt.Axis(title="ATM transactions", titleColor="#E68B39"),
    )
)

alt.layer(line2, line1).resolve_scale(y="independent").properties(
    width=800, height=300, title="Weekly ATM volume and count by potential customers"
)

alt.LayerChart(...)

Potential PWCB customers make around <b style="color:#E68B39">10k ATM transactions</b> and <b style="color:#5677A4">cash out around 900k EUR</b> 2 days around the visit to a store. This is around 90 EUR per transaction and roughly matches to the amounts current PWCB customers are doing.

If we could make 30% of potential customers to start using PWCB, we could expect they would transfer 60% of their transaction volume from ATM to PWCB because that's what our current PWCB customers are doing. This would potentially make around 1.8k transactions and 150k EUR weekly. 

Note that these calculations are rough and only orientational.

## 6. Customer subscriptions <a class="anchor" id="6"></a>

Let's look at subscription distribtution between potential and existing PWCB users. To contrast, the chart below also includes subscription distribution among all MAU customers.

Note: ATM here is customers who bought for >20 Euros and also cashed out <200 Euros 4 days around the visit to a store.

In [18]:
cst_prod = pd.concat([cst_product, cst_base], ignore_index=True)

cst_prod["cst_sum"] = cst_prod.groupby(["actions"])["count"].transform("sum")
cst_prod["cst_norm"] = cst_prod["count"] / cst_prod["cst_sum"]
bars = (
    alt.Chart(cst_prod)
    .mark_bar()
    .encode(
        x=alt.X("sum(count)", stack="normalize", title=None),
        y=alt.Y("actions", title=None),
        color=alt.Color("subscription", legend=alt.Legend(title="Subcsription")),
    )
    .properties(
        width=800, height=200, title="Existing/potential PWCB customers by subscription"
    )
)

text = (
    alt.Chart(cst_prod)
    .mark_text(dx=-20, dy=3, color="white", size=9)
    .encode(
        x=alt.X(
            "sum(count)",
            stack="normalize",
            title=None,
        ),
        y=alt.Y("actions", title=None),
        detail="subscription:N",
        text=alt.Text("cst_norm", format=".1%"),
        order=alt.Order("subscription", sort="descending"),
    )
)


bars + text

alt.LayerChart(...)

There is a significantly higher percentage of Flex customers among PWCB users. Premium account holder ratio using PWCB is slightly less than for MAUs, whereas it's slightly higher for ATM users.

## 7. Conclusion <a class="anchor" id="7"></a>

There are four main things to take out:

1. There are around 30k visits weekly to PWCB merchants when customers also cash out at ATMs. However most of them do transactions less than 20 EUR and would not be eligible for a PWCB transaction.

2. Potentially there are 5000 customers weekly who do around 9000 store visits with ATM per week. Instead of going to an ATM they could use PWCB as their amounts are fully eligible.

3. If we could make 30% of customers to use PWCB, potentially we could expect around 1.8k transactions and 150k EUR weekly to switch from ATM to PWCB.

4. A large portion of customers currently using PWCB are Flex customers 28.6% when there are only 2.9% of Flex users among MAUs. 

5. Out of potential customer pool 15.8% are premium customers who have higher ATM ceiling and could be pushed towards PWCB.


In [19]:
HTML(
    """
<script>
    code_show=true; 
    function code_toggle() {
        if (code_show){
            $('div.input').hide();
        } else {
            $('div.input').show();
            }
        code_show = !code_show
 
        $('div.output_subarea').css("text-align", "center"); 
        $('body').css("font-family", "Montserrat, sans-serif");
        $('h1').css("font-family", "Karla, sans-serif");
        $('h2').css("font-family", "Karla, sans-serif");
    } 
    $( document ).ready(code_toggle);
    
    </script>
    <form action="javascript:code_toggle()">
        <input type="submit" value="Click here to toggle on/off the raw code.">
    </form>
"""
)